# Import Libraries

In [ ]:
import os
import numpy as np
import bs4
import requests
import lxml

from pprint import pprint
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin, urlparse

# Parse all websites of the Manga

## colored

In [ ]:
html_text = requests.get('https://ww9.readonepiece.com/manga/one-piece-digital-colored-comics/', 'r').text
soup = BeautifulSoup(html_text, 'lxml')

cl_chapters = soup.find_all('div', class_ = 'bg-bg-secondary p-3 rounded mb-3 shadow')
cl_chapter_site = []
for chapter in cl_chapters:
    site = chapter.find('a', class_ = 'uppercase text-xs bg-text-success text-white font-bold py-1 px-2 rounded').attrs['href']
    cl_chapter_site.append(site)

cl_chapter_site.sort()
pprint(len(cl_chapter_site))

## grayscale

In [ ]:
html_text = requests.get('https://ww9.readonepiece.com/manga/one-piece/', 'r').text
soup = BeautifulSoup(html_text, 'lxml')

bw_chapters = soup.find_all('div', class_ = 'bg-bg-secondary p-3 rounded mb-3 shadow')
bw_chapter_site = []
for chapter in bw_chapters:
    site = chapter.find('a', class_ = 'uppercase text-xs bg-text-success text-white font-bold py-1 px-2 rounded').attrs['href']
    bw_chapter_site.append(site)

bw_chapter_site.sort()
pprint(len(bw_chapter_site))

In [ ]:
cl_chapters = []
bw_chapters = []
for i in tqdm(range(len(cl_chapter_site))):
    for j in range(len(bw_chapter_site)):
        if cl_chapter_site[i][-3:None] == bw_chapter_site[j][-3:None]:
            cl_chapters.append(cl_chapter_site[i])
            bw_chapters.append(bw_chapter_site[j])
        
cl_chapters.sort()
bw_chapters.sort()

In [ ]:
pprint(len(cl_chapters))
pprint(len(bw_chapters))

pprint(cl_chapters[-3:None])
pprint(bw_chapters[-3:None])

# Parsing all image urls from each chapter

## colored

In [ ]:
cl = []
for i in tqdm(range(len(cl_chapters)), "extracting images"):
    cl_imgs = []
    
    html_text = requests.get(cl_chapters[i], 'r').text
    soup = BeautifulSoup(html_text, 'lxml')

    all_imgs = soup.find_all('div', class_ = 'text-center')
    for img in all_imgs:
        site = img.find("img", class_ = 'mb-3 mx-auto js-page')
        if site is not None:
            cl_imgs.append(site['src'])
            
    cl = cl + cl_imgs
            
#     pprint(cl_imgs)
#     pprint(cl)
#     raise ValueError("done")

In [ ]:
print(len(cl))

## grayscale

In [ ]:
bw = []
for i in tqdm(range(len(bw_chapters)), "extracting images"):
    bw_imgs = []
    
    html_text = requests.get(bw_chapters[i], 'r').text
    soup = BeautifulSoup(html_text, 'lxml')

    all_imgs = soup.find_all('div', class_ = 'text-center')
    for img in all_imgs:
        site = img.find("img", class_ = 'mb-3 mx-auto js-page')
        if site is not None:
            bw_imgs.append(site['src'])    
            
    bw = bw + bw_imgs

In [ ]:
print(len(bw))

# Download the images

In [ ]:
# def download(url, pathname):
#     url = url.replace('\r', '')
    
#     # if path doesn't exist, make that path dir
#     if not os.path.isdir(pathname):
#         os.makedirs(pathname)

#     filename = os.path.join(pathname, url.split("/")[-1])
#     with open(filename, "wb") as f:
#         f.write(requests.get(url).content)

## colored

In [ ]:
pathname = './colored'
# if path doesn't exist, make that path dir
if not os.path.isdir(pathname):
    os.makedirs(pathname)
    
for i in tqdm(range(len(cl)), "downloading images"):
    url = cl[i].replace('\r', '')
        
    filename = os.path.join(pathname, str(i) + str('.png'))
    
    with open(filename, "wb") as f:
        f.write(requests.get(url).content)

## grayscale

In [ ]:
pathname = './grayscale'
# if path doesn't exist, make that path dir
if not os.path.isdir(pathname):
    os.makedirs(pathname)
    
for i in tqdm(range(len(bw)), "downloading images"):
    url = bw[i].replace('\r', '')
    
    filename = os.path.join(pathname, str(i) + str('.png'))
    
    with open(filename, "wb") as f:
        f.write(requests.get(url).content)